In [4]:
!pip install opencv-python
!pip install numpy==1.21.5
!pip install torch==2.0.0
!pip install ipython
!pip install torchvision
!pip install matplotlib==3.5.1
!pip install pandas==1.4.2
!pip install pyyaml
!pip install tqdm
!pip install seaborn


ERROR: Could not find a version that satisfies the requirement psutils (from versions: none)
ERROR: No matching distribution found for psutils


In [1]:
import cv2
import datetime
import torch
import os
import threading
import argparse
import keyboard

detect=False  # flag to indicate if object has been detected
halt=False    # flag to indicate if program should halt

In [2]:
def low_resolution(rtsp1,display):
    """
    Function to read low resolution video stream, detect objects, and set the detect flag if a person or car is detected.
    
    Args:
        rtsp1 (str): RTSP address for low resolution stream
        display (bool): Flag to indicate whether to display video stream
        
    Returns:
        None
    """
    cap = cv2.VideoCapture(rtsp1)
    if not cap.isOpened():
        raise Exception("Error: could not open video capture. Consider checking the RTSP address for low resolution")
        
    print("Detection Human and Vehicle")
    
    while(True):
        global detect, halt
        ret, frame = cap.read()
        
        # run object detection on the current frame
        results = model(frame)
        if display:
            cv2.imshow('frame',results.render()[0])
        
        # check if a person or car is detected, and set the detect flag accordingly
        try:
            if results.xyxy[0].tolist()[0][-1] == 0.0 or results.xyxy[0].tolist()[0][-1] == 2.0:
                detect=True
            else:
                detect=False
        except Exception as e:
            detect=False
            
        # display the video stream, if specified
        if display:
            if cv2.waitKey(1) & 0xFF == ord('q'):
                halt=True
                cap.release()
                cv2.destroyAllWindows()
                break
                
        # check if the user has pressed 'q' to end the program
        if keyboard.is_pressed('q'):
            halt=True
            cap.release()
            cv2.destroyAllWindows()
            break
    
def high_resolution(rtsp2,record_time,):
    """
    Function to read high resolution video stream, record video when the detect flag is set, and save the recording when
    a specified amount of time has elapsed since the object was detected.
    
    Args:
        rtsp2 (str): RTSP address for high resolution stream
        record_time (int): Number of seconds to record video after object is detected
        
    Returns:
        None
    """
    cap_full = cv2.VideoCapture(rtsp2)
    if not cap_full.isOpened():
        raise Exception("Error: could not open video capture. Consider checking the RTSP address for high resolution")
    frame_width = int(cap_full.get(3))
    frame_height = int(cap_full.get(4))
    fps = cap_full.get(cv2.CAP_PROP_FPS)
    print("Capturing high resolution video: {}x{} in {}".format(frame_width,frame_height,fps))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    recording=False
    count=1
    
    while(True):
        global detect, halt
        
        ret_full, frame_full = cap_full.read()
        
        # check if an object has been detected
        if detect:
            start_time = datetime.datetime.now()
            
            # if not already recording, start a new recording
            if not recording:
                print("Recording Started")
                output_path = os.path.join(output_dir, ('Camera_{}.mp4').format(count))
                out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
                recording=True
                count+=1
            
            # write the current frame to the recording
            out.write(frame_full)
        
        # If already recording, check if the recording timeout is exceeded
        elif recording:
            out.write(frame_full)
            if (datetime.datetime.now() - start_time).seconds > record_time:
                
                # Saving the recording as mp4
                print("Recording ('Camera_{}.mp4') Saved".format(count-1))
                recording= False
                out.release()
        
        # If 'q' is pressed end the program, using the global halt variable to achieve it
        if halt:
            print("Recording ('Camera_{}.mp4') Saved".format(count-1))
            cap_full.release()
            out.release()
            break
        

In [3]:
#Loading the YOLOv5m model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')
model.classes = [0, 2]  # person and car

#Creating directory to save the videos
output_dir = 'motion_detection'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# initialize the RTSP protocol
# RTSP protocol format
#"rtsp://[username]:[password]@[ip-address]/[url of stream]"
low_res='rtsp://admin:itachi@4@192.168.1.155:554/unicaststream/4'
high_res='rtsp://admin:itachi@4@192.168.1.155:554/unicaststream/1'
  
# If display is 1, a window with the object detection is shown
display=1
    
# It is the time between the object detected and the time at which it will stop recording
rec_timeout=20 # in seconds
    
# Using two threads - one for detecting object in low resolution and the other for saving high resolution video in local
t1 = threading.Thread(target=low_resolution,args=(low_res,display,))
t2 = threading.Thread(target=high_resolution,args=(high_res,rec_timeout,))
    
t1.start()
t2.start()
    
t1.join()
t2.join()

Using cache found in C:\Users\hariharan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-6 Python-3.9.16 torch-2.0.0+cpu CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


Detection Human and Vehicle
Capturing high resolution video: 1920x1080 in 25.0
Recording Started
Recording ('Camera_1.mp4') Saved
